# Artificial Neural Network

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

# goto the windows search terminal and type "Anaconda Prompt" and there type the command below 
#   conda install -c conda-forge tensorflow

In [3]:
tf.__version__

'2.10.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [4]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [7]:
#print(X)
#y
X.shape
y


array([1, 0, 1, ..., 1, 1, 0], dtype=int64)

In [52]:
print(y)
#y.shape

[1 0 1 ... 1 1 0]


### Encoding categorical data

Label Encoding the "Gender" column

In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

LabelEncoder Encode target labels with value between 0 and n_classes-1.

In [9]:
print(X)
#X.shape
#print(dataset.Geography.unique())


[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encoding the "Geography" column

In [10]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

OneHotEncoder encodes categorical features as a one-hot numeric array.

In [11]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


#Splitting the dataset into the Training set and Test set

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Feature Scaling

In [14]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X_test.shape

(2000, 12)

## Part 2 - Building the ANN

### Initializing the ANN

In [15]:
ann = tf.keras.models.Sequential()
#going to call sequential ANN, keras is library, models is module, sequential is class
# ann is instance of Sequential class, ann is a sequence of layers

### Adding the input layer and the first hidden layer

In [16]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
# add is a method in sequentail class
# dense class will create a fully connected layer
#all the features in the input file will be input neuron
#number of hidden layer is 6

### Adding the second hidden layer

In [17]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer

In [63]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [20]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [21]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 50)

Epoch 1/50
250/250 [==============================] - 0s 879us/step - loss: 0.3947 - accuracy: 0.1821
Epoch 2/50
250/250 [==============================] - 0s 896us/step - loss: 0.3923 - accuracy: 0.2435
Epoch 3/50
250/250 [==============================] - 0s 819us/step - loss: 0.3858 - accuracy: 0.3649
Epoch 4/50
250/250 [==============================] - 0s 861us/step - loss: 0.3856 - accuracy: 0.2134
Epoch 5/50
250/250 [==============================] - 0s 819us/step - loss: 0.3835 - accuracy: 0.2227
Epoch 6/50
250/250 [==============================] - 0s 819us/step - loss: 0.3845 - accuracy: 0.1548
Epoch 7/50
250/250 [==============================] - 0s 817us/step - loss: 0.3834 - accuracy: 0.1404
Epoch 8/50
250/250 [==============================] - 0s 882us/step - loss: 0.3800 - accuracy: 0.1686
Epoch 9/50
250/250 [==============================] - 0s 883us/step - loss: 0.3796 - accuracy: 0.1621
Epoch 10/50
250/250 [==============================] - 0s 882us/step - loss: 0.378

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer ?

**Solution**

In [24]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)
#false means customer will not leave the bank

1/1 [==============================] - 0s 16ms/step
[[False False False False False False]]


Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

### Predicting the Test set results

In [23]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 511us/step


ValueError: cannot reshape array of size 12000 into shape (2000,1)

### Making the Confusion Matrix

In [68]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1529   66]
 [ 206  199]]


0.864